In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
import ast

In [2]:
data = pd.read_csv('/content/combined_emails_with_natural_pii.csv')

In [3]:
data.head()

,email,type
0,Subject: Unvorhergesehener Absturz der Datenan...,Incident
1,Subject: Customer Support Inquiry\n\nSeeking i...,Request
2,Subject: Data Analytics for Investment\n\nI am...,Request
3,Subject: Krankenhaus-Dienstleistung-Problem\n\...,Incident
4,"Subject: Security\n\nDear Customer Support, I ...",Request


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   24000 non-null  object
 1   type    24000 non-null  object
dtypes: object(2)
memory usage: 375.1+ KB


In [5]:
data.isnull().sum()

,0
email,0
type,0


In [6]:
data['email'][1]

'Subject: Customer Support Inquiry\n\nSeeking information on digital strategies that can aid in brand growth and details on the available services. Looking forward to learning more to help our business grow My name is Elena Ivanova.. Thank you, and I look forward to hearing from you soon. You can reach me at fatima.farsi@help.com.'

In [7]:
data.head()

,email,type
0,Subject: Unvorhergesehener Absturz der Datenan...,Incident
1,Subject: Customer Support Inquiry\n\nSeeking i...,Request
2,Subject: Data Analytics for Investment\n\nI am...,Request
3,Subject: Krankenhaus-Dienstleistung-Problem\n\...,Incident
4,"Subject: Security\n\nDear Customer Support, I ...",Request


In [8]:
import pandas as pd
import spacy
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# 1. Define the PII masking function
def mask_pii_spacy(text):
    masked_text = text
    masked_entities = []

    regex_patterns = {
        "email": r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',
        "phone_number": r'\b(?:\+?(\d{1,3}))?[-. (]*(\d{3})[-. )]*(\d{3})[-. ]*(\d{4})\b',
        "credit_debit_no": r'\b(?:\d[ -]*?){13,16}\b',
        "aadhar_num": r'\d{4}\s\d{4}\s\d{4}',
        "cvv_no": r'\b\d{3,4}\b',
        "expiry_no": r'\b(0[1-9]|1[0-2])/?([0-9]{2})\b',
        "dob": r'\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b'
    }

    for entity_type, pattern in regex_patterns.items():
        for match in re.finditer(pattern, masked_text):
            start, end = match.span()
            original_entity = match.group(0)
            masked_text = masked_text[:start] + f"[{entity_type}]" + masked_text[end:]
            masked_entities.append({
                "entity": original_entity,
                "classification": entity_type,
                "position": [start, end]
            })
            offset = len(f"[{entity_type}]") - len(original_entity)
            for i in range(len(masked_entities)-1):
                if masked_entities[i]["position"][0] > start:
                    masked_entities[i]["position"] = [
                        masked_entities[i]["position"][0] + offset,
                        masked_entities[i]["position"][1] + offset
                    ]

    doc = nlp(masked_text)

    spacy_entity_mapping = {
        "PERSON": "full_name",
        "ORG": "organization",
        "GPE": "location",
        "DATE": "dob"
    }

    for ent in doc.ents:
        if ent.label_ in spacy_entity_mapping:
            start, end = ent.start_char, ent.end_char
            original_entity = ent.text
            label = spacy_entity_mapping[ent.label_]
            masked_text = masked_text[:start] + f"[{label}]" + masked_text[end:]
            masked_entities.append({
                "entity": original_entity,
                "classification": label,
                "position": [start, end]
            })
            offset = len(f"[{label}]") - len(original_entity)
            for i in range(len(masked_entities)-1):
                if masked_entities[i]["position"][0] > start:
                    masked_entities[i]["position"] = [
                        masked_entities[i]["position"][0] + offset,
                        masked_entities[i]["position"][1] + offset
                    ]

    return masked_text, masked_entities

# 2. Define the text preprocessing function
def text_preprocessing(data, text_column='masked_text'):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    def clean_text(text):
        text = text.lower()

        # Step 1: Extract [masked_tokens] to preserve
        tokens = re.findall(r'\[[^\]]+\]', text)
        placeholders = {token: f"__TOKEN_{i}__" for i, token in enumerate(tokens)}

        # Step 2: Temporarily replace them
        for token, placeholder in placeholders.items():
            text = text.replace(token, placeholder)

        # Step 3: Remove unwanted characters from rest of text
        text = re.sub(r'[^a-z\s]', '', text)

        # Step 4: Restore original tokens
        for token, placeholder in placeholders.items():
            text = text.replace(placeholder, token)

        # Step 5: Tokenize, remove stopwords, and lemmatize
        words = text.split()
        words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words and len(word) > 1]
        return " ".join(words)

    data[text_column] = data[text_column].apply(clean_text)
    return data

# 4. Apply PII Masking
def apply_masking(row):
    masked_text, masked_entities = mask_pii_spacy(row["email"])
    return pd.Series([masked_text, masked_entities])

data[["masked_text", "masked_entities"]] = data.apply(apply_masking, axis=1)

# 5. Apply Text Preprocessing on masked text
data = text_preprocessing(data, text_column='masked_text')

# 6. Final Data
print(data)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


                                                   email      type  \
0      Subject: Unvorhergesehener Absturz der Datenan...  Incident   
1      Subject: Customer Support Inquiry\n\nSeeking i...   Request   
2      Subject: Data Analytics for Investment\n\nI am...   Request   
3      Subject: Krankenhaus-Dienstleistung-Problem\n\...  Incident   
4      Subject: Security\n\nDear Customer Support, I ...   Request   
...                                                  ...       ...   
23995  Subject: Problem mit der HP DeskJet 3755 WLAN-...   Problem   
23996  Subject: Problemas com a impressora HP DeskJet...  Incident   
23997  Subject: Problema urgente con el envío\n\nEsti...  Incident   
23998  Subject: \n\nCher Service Client, nous rencont...  Incident   
23999  Subject: Overcharge on yearly Microsoft Office...   Problem   

                                             masked_text  \
0      subject unvorhergesehener attform brach unerwa...   
1      subject customer support inquiry

In [9]:
data['email'][1]

'Subject: Customer Support Inquiry\n\nSeeking information on digital strategies that can aid in brand growth and details on the available services. Looking forward to learning more to help our business grow My name is Elena Ivanova.. Thank you, and I look forward to hearing from you soon. You can reach me at fatima.farsi@help.com.'

In [10]:
print(data.columns)

Index(['email', 'type', 'masked_text', 'masked_entities'], dtype='object')


In [11]:
data.head()

,email,type,masked_text,masked_entities
0,Subject: Unvorhergesehener Absturz der Datenan...,Incident,subject unvorhergesehener attform brach unerwa...,"[{'entity': 'janesmith@company.com', 'classifi..."
1,Subject: Customer Support Inquiry\n\nSeeking i...,Request,subject customer support inquiry seeking infor...,"[{'entity': 'fatima.farsi@help.com', 'classifi..."
2,Subject: Data Analytics for Investment\n\nI am...,Request,subject contacting request information data an...,"[{'entity': 'liuwei@business.cn', 'classificat..."
3,Subject: Krankenhaus-Dienstleistung-Problem\n\...,Incident,subject ein mediendatensperrbten zugriffes auf...,"[{'entity': 'fatima.farsi@help.com', 'classifi..."
4,"Subject: Security\n\nDear Customer Support, I ...",Request,subject security dear customer support reachin...,"[{'entity': 'fatima.farsi@help.com', 'classifi..."


In [12]:
data['masked_text'][1]

'subject customer support inquiry seeking information digital strategy aid brand growth detail available service looking forward learning help business grow name thank look forward hearing soon reach'

In [13]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [14]:
def add_keyword_features(df, text_column="masked_text"):
    request_keywords = ["please", "kindly", "request", "could you", "would you"]
    problem_keywords = ["issue", "error", "problem", "failed", "unable"]
    change_keywords = ["change", "update", "modify", "replace", "upgrade"]

    def check_keywords(text, keywords):
        return int(any(word in text.lower() for word in keywords))

    df['has_request_words'] = df[text_column].apply(lambda x: check_keywords(x, request_keywords))
    df['has_problem_words'] = df[text_column].apply(lambda x: check_keywords(x, problem_keywords))
    df['has_change_words'] = df[text_column].apply(lambda x: check_keywords(x, change_keywords))

    return df


In [15]:
from sentence_transformers import SentenceTransformer

# Load sentence transformer model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode masked_text into sentence embeddings
email_embeddings = sbert_model.encode(data['masked_text'].tolist(), batch_size=32, show_progress_bar=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/750 [00:00<?, ?it/s]

In [16]:
x_final = email_embeddings

In [17]:
data.head()

,email,type,masked_text,masked_entities
0,Subject: Unvorhergesehener Absturz der Datenan...,Incident,subject unvorhergesehener attform brach unerwa...,"[{'entity': 'janesmith@company.com', 'classifi..."
1,Subject: Customer Support Inquiry\n\nSeeking i...,Request,subject customer support inquiry seeking infor...,"[{'entity': 'fatima.farsi@help.com', 'classifi..."
2,Subject: Data Analytics for Investment\n\nI am...,Request,subject contacting request information data an...,"[{'entity': 'liuwei@business.cn', 'classificat..."
3,Subject: Krankenhaus-Dienstleistung-Problem\n\...,Incident,subject ein mediendatensperrbten zugriffes auf...,"[{'entity': 'fatima.farsi@help.com', 'classifi..."
4,"Subject: Security\n\nDear Customer Support, I ...",Request,subject security dear customer support reachin...,"[{'entity': 'fatima.farsi@help.com', 'classifi..."


In [18]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 27.3 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow

In [28]:
!pip install tensorflow==2.11.0
!pip install keras==2.11.0


ERROR: Could not find a version that satisfies the requirement tensorflow==2.11.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0)
ERROR: No matching distribution found for tensorflow==2.11.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.7 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.8.0
    Uninstalling keras-3.8.0:
      Successfully uninstalled keras-3.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires keras>=3.5.0, but you have keras 2.11.0 which is incompatible.
tensorflow-text 2.18.1 requires tensorflo

In [30]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score

# Assuming 'data' and 'x_final' are already defined
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['type'])

# Target variable
#y = data['type']
print(y)
data.head()
# Train-test split
x_train, x_test, y_train, y_test = train_test_split(x_final, y, test_size=0.2, random_state=42, stratify=y)

# Reshaping input data for GRU (needs 3D input shape: [samples, time steps, features])
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))  # 1 time step for each sample
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))

# Define the GRU model
model = Sequential()

# GRU layer (input_shape = (time_steps, features))
model.add(GRU(128, input_shape=(x_train.shape[1], x_train.shape[2]), activation='relu'))
model.add(Dropout(0.3))  # Dropout to prevent overfitting

# Add hidden layers (dense layer after GRU)
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))

# Output layer (number of classes)
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test))

# Predictions
y_pred_prob = model.predict(x_test)
y_pred = tf.argmax(y_pred_prob, axis=1)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
print(f"Precision: {precision}")
print(f"Accuracy: {accuracy}")


[1 3 3 ... 1 1 2]
Epoch 1/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.5803 - loss: 0.9756 - val_accuracy: 0.6898 - val_loss: 0.6850
Epoch 2/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.6887 - loss: 0.6926 - val_accuracy: 0.6952 - val_loss: 0.6552
Epoch 3/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.6993 - loss: 0.6576 - val_accuracy: 0.7090 - val_loss: 0.6394
Epoch 4/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.7040 - loss: 0.6304 - val_accuracy: 0.7121 - val_loss: 0.6247
Epoch 5/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.7185 - loss: 0.6056 - val_accuracy: 0.7167 - val_loss: 0.6194
Epoch 6/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.7260 - loss: 0.5915 - val_accuracy: 0.7208 - val_loss: 0.6139
Epoch 7/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7295 - loss: 0.5742 - val_accuracy: 0.7229 - val_loss: 0.6153
Epoch 8/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.7323 - loss: 0.5692

In [20]:
data.head()

,email,type,masked_text,masked_entities
0,Subject: Unvorhergesehener Absturz der Datenan...,Incident,subject unvorhergesehener attform brach unerwa...,"[{'entity': 'janesmith@company.com', 'classifi..."
1,Subject: Customer Support Inquiry\n\nSeeking i...,Request,subject customer support inquiry seeking infor...,"[{'entity': 'fatima.farsi@help.com', 'classifi..."
2,Subject: Data Analytics for Investment\n\nI am...,Request,subject contacting request information data an...,"[{'entity': 'liuwei@business.cn', 'classificat..."
3,Subject: Krankenhaus-Dienstleistung-Problem\n\...,Incident,subject ein mediendatensperrbten zugriffes auf...,"[{'entity': 'fatima.farsi@help.com', 'classifi..."
4,"Subject: Security\n\nDear Customer Support, I ...",Request,subject security dear customer support reachin...,"[{'entity': 'fatima.farsi@help.com', 'classifi..."


In [21]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sentence_transformers import SentenceTransformer

def predict_single_email(email_text, model, label_encoder):
    # 1. Create a dummy DataFrame
    email_df = pd.DataFrame({'email': [email_text]})

    # 2. Apply masking
    email_df[["masked_text", "masked_entities"]] = email_df.apply(apply_masking, axis=1)

    # 3. Preprocess text
    email_df = text_preprocessing(email_df, text_column="masked_text")

    # 4. Sentence Embedding
    sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
    sentence_embeddings = sentence_model.encode(email_df["masked_text"].tolist())

    # 5. Reshape for LSTM/GRU (samples, time_steps, features)
    final_features = sentence_embeddings.reshape((sentence_embeddings.shape[0], 1, sentence_embeddings.shape[1]))

    # 6. Predict
    y_pred_prob = model.predict(final_features)  # shape = (1, 4) --> probabilities for 4 classes
    y_pred_class = np.argmax(y_pred_prob, axis=1)  # shape = (1,) --> class ID like 0, 1, 2, 3

    # 7. Convert predicted number to label
    y_pred_class = y_pred_class.reshape(-1)  # Ensure it is an array
    predicted_label = label_encoder.inverse_transform(y_pred_class)[0]

    return {
        "predicted_category": str(predicted_label)
    }


In [22]:
email_text = """Request to Update My Contact Information

Hi,

I recently changed my phone number and email address. Could you please update my records with the following details?

New Email: john.doe@example.com
New Phone Number: +1 (555) 123-4567

Let me know if you need anything else from my side.

Thanks,
John Doe
"""

result = predict_single_email(email_text, model=model, label_encoder=label_encoder)

print(result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
{'predicted_category': 'Change'}


In [32]:
!pip install pyyaml h5py  # Install necessary dependencies

model.save("GRU_Model.h5")  # Save using TensorFlow's native save method
# Load your current .h5 model
from tensorflow.keras.models import load_model

model = load_model("GRU_Model.h5")

# Re-save using TensorFlow's SavedModel format (version-friendly)
# Use the export() method for saving as a SavedModel
tf.saved_model.save(model, "GRU_Model_SavedFormat")

Cause: Unable to locate the source code of <function TFLayer._default_save_signature.<locals>.serving_default at 0x7ed405a6f060>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function TFLayer._default_save_signature.<locals>.serving_default at 0x7ed405a6f060>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function TFLayer._default_save_signature.<locals>.serving_default at 0x7ed3d61e80e0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function TFLayer._default_save_signature.<locals>.serving_default at 0x7ed3d61e80e0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [27]:
import json
label_encoder_dict = {"classes_": label_encoder.classes_.tolist()}
with open('label_encoder.json', 'w') as f:
    json.dump(label_encoder_dict, f)